<h1>CS 41344 - Natural Language Processing</h1>
<h2 style = 'color:yellow'>Assignment - 6: Deep Learning</h2>

<h3 style = 'color:lightgreen'>Assignment - 6.0 Import Libraries</h3>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

<h4 style = 'color:green'>6.1 Calculate Activation Function</h4>

In [2]:
def calculate_activation(perceptron_value: tf.Tensor, activation_type: str) -> tf.Tensor:
    """
    Calculate the activation of a perceptron value based on the activation type.
    
    Parameters
    ----------
    perceptron_value : tf.Tensor
        The value of the perceptron.
    activation_type : str
        The type of activation to use.
    
    Returns
    -------
    tf.Tensor
        The activated value of the perceptron.
    """

    if activation_type == "ReLu":
        return tf.maximum(0, perceptron_value)
    elif activation_type == "Sigmoid":
        return 1 / (1 + tf.exp(-perceptron_value))
    elif activation_type == "Tanh":
        return (tf.exp(perceptron_value) - tf.exp(-perceptron_value)) / (tf.exp(perceptron_value) + tf.exp(-perceptron_value))
    elif activation_type == "Linear":
        return perceptron_value

In [4]:
inputs = tf.constant([0.1, 0.2])
weights_1 = tf.constant([0.15, 0.05])
bias_1 = tf.constant(0.33)

# Using tensordot for dot product
layer_1_output = tf.tensordot(inputs, weights_1, axes=1) + bias_1
# print(f"Layer 1 Output: {layer_1_output.numpy()}")

layer_1_activation = calculate_activation(layer_1_output, "Sigmoid")
# print(f"Layer 1 Activation (Sigmoid): {layer_1_activation.numpy()}")

layer_2_weight = tf.constant(0.36)
layer_2_bias = tf.constant(0.56)

layer_2_output = layer_1_activation * layer_2_weight + layer_2_bias

layer_2_activation = calculate_activation(layer_2_output, "Sigmoid")
# print(f"Layer 2 Activation (Sigmoid): {layer_2_activation.numpy()}")

In [5]:
activation_functions = ["Sigmoid", "Linear", "Tanh", "ReLu"]
actF_layer1 = []
actF_Layer2 = []
for _ in activation_functions:
    actF_layer1.append(calculate_activation(layer_1_output, _).numpy())
    actF_Layer2.append(calculate_activation(layer_2_output, _).numpy())
    
df = pd.DataFrame({
    "Activation Function": activation_functions,
    "Layer 1 Activation": actF_layer1,
    "Layer 2 Activation": actF_Layer2
})
df

,Activation Function,Layer 1 Activation,Layer 2 Activation
0,Sigmoid,0.587830,0.683871
1,Linear,0.355000,0.771619
2,Tanh,0.340802,0.647870
3,ReLu,0.355000,0.771619


<h4 style = 'color:green'>6.2 Calculate Gradients</h4>

In [6]:
def calculate_gradient(perceptron_value: tf.Tensor, activation_type: str) -> tf.Tensor:

    """
    This function calculates the gradient of the perceptron value based on the activation type.

    Parameters
    ----------
    perceptron_value : tf.Tensor
        The value of the perceptron.
    activation_type : str
        The type of activation to use.
    
    Returns
    -------
    tf.Tensor
        The gradient of the perceptron value.
    """

    if activation_type == "ReLu":
        return tf.where(perceptron_value < 0, 0, 1)
    elif activation_type == "Sigmoid":
        return calculate_activation(perceptron_value, "Sigmoid") * (1 - calculate_activation(perceptron_value, "Sigmoid"))
    elif activation_type == "Tanh":
        return 1 - tf.square(calculate_activation(perceptron_value, "Tanh"))
    elif activation_type == "Linear":
        return 1

In [7]:
v1 = -4.0
v2 = 0.5
v3 = 4.0

activation_functions = ["Sigmoid", "Tanh", "ReLu"]
gradF_v1 = []
gradF_v2 = []
gradF_v3 = []

for _ in activation_functions:
    gradF_v1.append(calculate_gradient(v1, _).numpy())
    gradF_v2.append(calculate_gradient(v2, _).numpy())
    gradF_v3.append(calculate_gradient(v3, _).numpy())

df = pd.DataFrame({
    "Activation Function": activation_functions,
    "v1 Gradient": gradF_v1,
    "v2 Gradient": gradF_v2,
    "v3 Gradient": gradF_v3
})

df

,Activation Function,v1 Gradient,v2 Gradient,v3 Gradient
0,Sigmoid,0.017663,0.235004,0.017663
1,Tanh,0.001341,0.786448,0.001341
2,ReLu,0.000000,1.000000,1.000000


<h4 style = 'color:green'>6.3 Calculate Softmax</h4>

In [8]:
def softmax(x):

    """
    This function calculates the softmax of a tensor.
    
    Parameters
    ----------
    x : tf.Tensor
        The input tensor.
    
    Returns
    -------
    tf.Tensor
        The softmax of the input tensor.
    """

    return tf.exp(x) / tf.reduce_sum(tf.exp(x))

In [9]:
input_softmax_1 = tf.constant([2.3, 1.2, 0.3, 0.0])
input_softmax_2 = tf.constant([1.9, 1.7, 2.6, 0.2, 1.3])

softmax_output_1 = softmax(input_softmax_1)
softmax_output_2 = softmax(input_softmax_2)

data = {
    "Input": [input_softmax_1.numpy(), input_softmax_2.numpy()],
    "Softmax Output": [np.round(softmax_output_1.numpy(), 4), np.round(softmax_output_2.numpy(), 4)]
}

df = pd.DataFrame(data)
df

,Input,Softmax Output
0,"[2.3, 1.2, 0.3, 0.0]","[0.6376, 0.2122, 0.0863, 0.0639]"
1,"[1.9, 1.7, 2.6, 0.2, 1.3]","[0.2191, 0.1794, 0.4412, 0.04, 0.1202]"


<h4 style = 'color:green'>6.4 Calculate Cross-Entropy Loss</h4>

In [10]:
def cross_entropy_loss(target: tf.Tensor, compValue: tf.Tensor) -> tf.Tensor:

    """
    This function calculates the cross-entropy loss between the target and the computed value.
    
    Parameters
    ----------
    target : tf.Tensor
        The target tensor.
    compValue : tf.Tensor
        The computed value tensor.
    
    Returns
    -------
    tf.Tensor
        The cross-entropy loss between the target and the computed value.
    """

    loss =  - ((target * np.log(compValue) + (1 - target) * np.log(1 - compValue)))
    return loss

In [11]:
data = {
    "Target [0]": [0, 0, 0, 0, 0],
    "Computed Value": [0.95, 0.8, 0.6, 0.4, 0.1],
    "Target [1]": [1, 1, 1, 1, 1],
}

df = pd.DataFrame(data)
df

,Target [0],Computed Value,Target [1]
0,0,0.95,1
1,0,0.80,1
2,0,0.60,1
3,0,0.40,1
4,0,0.10,1


In [12]:
df['Cross Entropy Loss [0]'] = cross_entropy_loss(df['Target [0]'], df['Computed Value'])
df['Cross Entropy Loss [1]'] = cross_entropy_loss(df['Target [1]'], df['Computed Value'])
df

,Target [0],Computed Value,Target [1],Cross Entropy Loss [0],Cross Entropy Loss [1]
0,0,0.95,1,2.995732,0.051293
1,0,0.80,1,1.609438,0.223144
2,0,0.60,1,0.916291,0.510826
3,0,0.40,1,0.510826,0.916291
4,0,0.10,1,0.105361,2.302585


<h4 style = 'color:green'>6.5 ArgMax Function</h4>

In [13]:
a = [
    [5, 2, 3],
    [26, 56, 92],
    [3, 0, 26]
]
a

[[5, 2, 3], [26, 56, 92], [3, 0, 26]]

In [14]:
a1 = tf.argmax(a, axis=0).numpy()
a2 = tf.argmax(a, axis=1).numpy()

print(f"Argmax (axis=0): {a1}")
print(f"Argmax (axis=1): {a2}")

Argmax (axis=0): [1 1 1]
Argmax (axis=1): [0 2 2]


<h4 style = 'color:green'>6.6 XOR Function</h4>

In [28]:
inputs = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
])

In [17]:
weights_1 = np.array([
    [-4, -6, -5],
    [3, 6, 4]
], dtype=np.float32)

bias_1 = np.array([-2, 3, -2], dtype=np.float32)

weights_2 = np.array([
    [5],
    [-9],
    [7]
], dtype=np.float32)

bias_2 = np.array([4], dtype=np.float32)

In [29]:
# Calculate hidden layer output
hidden_layer_input = np.dot(inputs, weights_1) + bias_1
hidden_layer_output = calculate_activation(hidden_layer_input, "Sigmoid")

In [31]:
# Calculate output layer output
output_layer_input = np.dot(hidden_layer_output, weights_2) + bias_2
final_output = calculate_activation(output_layer_input, "Sigmoid")
final_output

<tf.Tensor: shape=(4, 1), dtype=float64, numpy=
array([[0.04137861],
       [0.97319269],
       [0.99201349],
       [0.0179147 ]])>

In [36]:
df = pd.DataFrame({
    'Input 1': inputs[:, 0],
    'Input 2': inputs[:, 1],
    'True Output': [0, 1, 1, 0]
})
df['Computed Output'] = final_output
df['Error'] = (df['Computed Output'] - df['True Output']) ** 2
df

,Input 1,Input 2,True Output,Computed Output,Error
0,0,0,0,0.041379,0.001712
1,1,0,1,0.973193,0.000719
2,0,1,1,0.992013,0.000064
3,1,1,0,0.017915,0.000321
